In [ ]:
# train_and_proto.py
import os
import json
from pathlib import Path
from collections import defaultdict, Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import timm
from helpers.helper import Helper
from torch.optim.lr_scheduler import CosineAnnealingLR

# =========================
# CONFIG
# =========================
#DATA_DIR = 'C:\\Identification dataset\\Cleaned_identification_dataset (August 2025)'
DATA_DIR = 'C:\\Identification dataset\\KNP_identification_dataset_nighttime'
BATCH_SIZE = 16
NUM_EPOCHS = 100
#SAVE_ROOT = Path("identification_models_Sumi") / "SUMI_CONVNEXT_2"
SAVE_ROOT = Path("identification_models_KNP") / "KNP_CONVNEXT_Nighttime"

def increment_path(path: Path, mkdir=True) -> Path:
    base = Path(path)
    if not base.exists():
        if mkdir:
            base.mkdir(parents=True, exist_ok=True)
        return base
    i = 1
    while True:
        p = Path(f"{str(base)}_{i:02d}")
        if not p.exists():
            if mkdir:
                p.mkdir(parents=True, exist_ok=True)
            return p
        i += 1

SAVE_PATH = increment_path(SAVE_ROOT, mkdir=True)
MODEL_SAVE_PATH = SAVE_PATH / "best_model.pth"
LAST_MODEL_SAVE_PATH = SAVE_PATH / "last_model.pth"
LABEL_JSON_PATH = SAVE_PATH / "class_mappings.json"
CHECKPOINT_DIR = SAVE_PATH / "checkpoints"
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# TRANSFORMS
# =========================
train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# =========================
# DATA
# =========================\

train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'Training'), transform=train_transform)
val_dataset   = datasets.ImageFolder(os.path.join(DATA_DIR, 'Validation'), transform=val_transform)

#train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
#val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, pin_memory= True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

# Save class index mapping to JSON (idx -> class_name)
class_to_idx = train_dataset.class_to_idx                 # {'class_name': idx}
idx_to_class = {v: k for k, v in class_to_idx.items()}    # {idx: 'class_name'}

with open(LABEL_JSON_PATH, 'w', encoding='utf-8') as f:
    json.dump(idx_to_class, f, indent=4, ensure_ascii=False)
print(f"✅ Saved label mapping to {LABEL_JSON_PATH}")

NUM_CLASSES = len(class_to_idx)

# =========================
# MODEL / OPTIM / SCHED
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('convnextv2_base.fcmae_ft_in22k_in1k',
                          pretrained=True, num_classes=NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# =========================
# LR SCALED BY BATCH SIZE
# =========================
BASE_BATCH_SIZE = 16   # reference batch size
head_lr_base = 1e-3
backbone_lr_base = 1e-4

# Linear scaling rule
scale_factor = BATCH_SIZE / BASE_BATCH_SIZE
head_lr = head_lr_base * scale_factor
backbone_lr = backbone_lr_base * scale_factor

print(f"⚡ Using batch_size={BATCH_SIZE}, head_lr={head_lr_base:.2e}, backbone_lr={backbone_lr_base:.2e}")

optimizer = torch.optim.AdamW([
    {"params": model.get_classifier().parameters(), "lr": 1e-3},  # classification head
    {"params": [p for n, p in model.named_parameters() if "head" not in n], "lr": 1e-4},  # rest
], weight_decay=0.01)

scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# =========================
# TRAIN / EVAL
# =========================
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    loop = tqdm(dataloader, desc="Training", leave=False)
    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad(set_to_none=True)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    return running_loss / len(dataloader)

@torch.no_grad()
def evaluate(model, dataloader, device):
    model.eval()
    preds, trues = [], []
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    for inputs, labels in tqdm(dataloader, desc="Validation", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        preds.extend(predicted.cpu().tolist())
        trues.extend(labels.cpu().tolist())

        for t, p in zip(labels.cpu().tolist(), predicted.cpu().tolist()):
            class_total[t] += 1
            if t == p:
                class_correct[t] += 1

    acc = accuracy_score(trues, preds)
    return acc, class_correct, class_total

# =========================
# EMBEDDINGS / PROTOTYPES
# =========================
@torch.no_grad()
def get_embeddings(model, x):
    """
    Get L2-normalized pre-logits embeddings from ConvNeXtV2.
    timm convnextv2: forward_features -> forward_head(pre_logits=True).
    """
    feats = model.forward_features(x)                  # pre-classifier features
    emb  = model.forward_head(feats, pre_logits=True)  # pooled pre-logits
    return F.normalize(emb, dim=1)                     # L2-normalize

@torch.no_grad()
def build_class_prototypes(model, data_dir, class_to_idx, device, batch_size=64):
    # Non-augmented transform for stable prototypes
    proto_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]),
    ])
    ds = datasets.ImageFolder(Path(data_dir) / "Training", transform=proto_transform)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False)

    buckets = defaultdict(list)  # class_idx -> [feat]
    model.eval()
    for imgs, labels in tqdm(dl, desc="Build Prototypes", leave=False):
        imgs = imgs.to(device)
        feats = get_embeddings(model, imgs)  # [B,D]
        for f, y in zip(feats, labels):
            buckets[int(y.item())].append(f)

    labels = []
    protos = []
    for cls_name, cls_idx in sorted(class_to_idx.items(), key=lambda kv: kv[1]):
        feats_list = buckets[cls_idx]
        if len(feats_list) == 0:
            continue
        feats = torch.stack(feats_list, dim=0)     # [N,D]
        proto = F.normalize(feats.mean(dim=0), dim=0)
        protos.append(proto)
        labels.append(cls_name)

    prototypes = torch.stack(protos, dim=0)        # [K,D], L2-normalized
    return prototypes, labels

@torch.no_grad()
def tune_global_threshold(model, prototypes, val_loader, device, target_tpr=0.95):
    """
    Choose a cosine threshold τ so ~target_tpr of *correct* validation samples are accepted.
    Does not require unknown data.
    """
    sims_for_correct = []

    model.eval()
    for imgs, labels in tqdm(val_loader, desc="Tune τ", leave=False):
        imgs = imgs.to(device)
        feats = get_embeddings(model, imgs)             # [B,D]
        sims = feats @ prototypes.T                     # [B,K]
        max_sims, pred_idx = sims.max(dim=1)
        correct = pred_idx.cpu() == labels.cpu()
        sims_for_correct.extend(ms for ms, ok in zip(max_sims.cpu().tolist(), correct) if ok)

    if len(sims_for_correct) == 0:
        return 0.7  # fallback

    sims_tensor = torch.tensor(sims_for_correct)
    miss_rate = 1 - float(target_tpr)
    q = 100 * miss_rate
    tau = torch.quantile(sims_tensor, q/100.0).item()
    return max(0.4, min(0.95, tau))

# =========================
# INFERENCE HELPER
# =========================
class ProtoInfer:
    def __init__(self, model, device, proto_ckpt_path):
        self.model = model.eval().to(device)
        self.device = device
        ckpt = torch.load(proto_ckpt_path, map_location=device)

        self.prototypes = ckpt["prototypes"].to(device)   # [K,D] L2-normalized
        self.proto_labels = ckpt["proto_labels"]          # list[str]
        self.tau = float(ckpt["tau"])
        emb_dim = ckpt.get("embedding_dim", None)

        # Optional safety check for embedding dimension consistency
        if emb_dim is not None and self.prototypes.shape[1] != emb_dim:
            raise ValueError(f"Embedding dim mismatch: ckpt={emb_dim}, prototypes D={self.prototypes.shape[1]}")

    @torch.no_grad()
    def predict_stack(self, images_tensors, tau=None):
        """
        images_tensors: list[Tensor[C,H,W]] transformed like val_transform
        returns: list[[class_name, count], ...] (includes 'unknown')
        """
        if tau is None:
            tau = self.tau
        x = torch.stack(images_tensors).to(self.device)  # [B,C,H,W]
        feats = get_embeddings(self.model, x)            # [B,D]
        sims  = feats @ self.prototypes.T                # [B,K]
        max_sims, idxs = sims.max(dim=1)

        labels = []
        for s, i in zip(max_sims.tolist(), idxs.tolist()):
            if s >= tau:
                labels.append(self.proto_labels[i])
            else:
                labels.append("unknown")

        counts = Counter(labels)
        return [[k, v] for k, v in counts.items()]

# =========================
# TRAINING LOOP + EXPORT
# =========================
def main():
    print(f"📁 Save dir: {SAVE_PATH}")
    best_acc = 0.0

    for epoch in range(NUM_EPOCHS):
        print(f"\n📚 Epoch {epoch + 1}/{NUM_EPOCHS}")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_acc, class_correct, class_total = evaluate(model, val_loader, device)

        print(f"📈 Loss: {train_loss:.4f} - Val Acc: {val_acc:.4f}")
        print("📊 Per-Class Accuracy:")
        for class_idx, total in class_total.items():
            correct = class_correct[class_idx]
            acc_cls = correct / total if total > 0 else 0.0
            class_name = idx_to_class[class_idx]
            print(f"  {class_name:<20}: {acc_cls:.4f} ({correct}/{total})")

        # Save best model
        if val_acc >= best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"✅ Best model saved at {MODEL_SAVE_PATH}")

        # Always save last
        torch.save(model.state_dict(), LAST_MODEL_SAVE_PATH)

        # Periodic checkpoints
        if (epoch + 1) % 10 == 0:
            ckpt_path = CHECKPOINT_DIR / f"checkpoint_epoch_{epoch+1}.pth"
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_acc': best_acc
            }, ckpt_path)
            print(f"💾 Checkpoint saved at {ckpt_path}")

        scheduler.step()

    # --------- Build prototypes + tune τ (using best model) ---------
    print("\n🧩 Building prototypes + tuning τ on validation set...")
    model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
    model.eval()

    prototypes, proto_labels = build_class_prototypes(
        model=model,
        data_dir=DATA_DIR,
        class_to_idx=class_to_idx,
        device=device,
        batch_size=64
    )
    print(f"✅ Prototypes: {tuple(prototypes.shape)} (classes={len(proto_labels)})")

    tau = tune_global_threshold(model, prototypes.to(device), val_loader, device, target_tpr=0.95)
    print(f"✅ Tuned cosine threshold τ = {tau:.3f}")

    # 🔐 Save everything needed for identification inference
    torch.save({
        "prototypes": prototypes.cpu(),           # [K,D], L2-normalized
        "proto_labels": proto_labels,             # list[str]
        "tau": float(tau),                        # global threshold
        "embedding_dim": int(prototypes.shape[1]) # D (sanity check)
    }, SAVE_PATH / "prototypes.pt")
    print(f"💾 Saved prototypes + τ to {SAVE_PATH/'prototypes.pt'}")

if __name__ == "__main__":
    main()


✅ Saved label mapping to identification_models_KNP\KNP_CONVNEXT_Nighttime_01\class_mappings.json
⚡ Using batch_size=16, head_lr=1.00e-03, backbone_lr=1.00e-04
📁 Save dir: identification_models_KNP\KNP_CONVNEXT_Nighttime_01

📚 Epoch 1/100


📈 Loss: 1.3061 - Val Acc: 0.9944
📊 Per-Class Accuracy:
  1374                : 1.0000 (76/76)
  3875                : 1.0000 (84/84)
  4142                : 1.0000 (46/46)
  4161                : 1.0000 (52/52)
  4307                : 1.0000 (35/35)
  4381                : 1.0000 (59/59)
  4413                : 1.0000 (44/44)
  4538                : 1.0000 (69/69)
  4548                : 1.0000 (50/50)
  4554                : 0.9857 (69/70)
  4598                : 1.0000 (70/70)
  4608                : 1.0000 (69/69)
  4627                : 1.0000 (54/54)
  4628                : 1.0000 (13/13)
  4773                : 1.0000 (33/33)
  4781                : 1.0000 (29/29)
  4783                : 1.0000 (97/97)
  4791                : 0.9889 (89/90)
  4815                : 1.0000 (129/129)
  4816                : 1.0000 (81/81)
  4826                : 1.0000 (92/92)
  4838                : 1.0000 (70/70)
  4845                : 0.9508 (58/61)
  4856                : 1.0000 (66/66)
  4872 

📈 Loss: 0.7057 - Val Acc: 0.9369
📊 Per-Class Accuracy:
  1374                : 0.9868 (75/76)
  3875                : 0.9881 (83/84)
  4142                : 1.0000 (46/46)
  4161                : 0.9808 (51/52)
  4307                : 0.9714 (34/35)
  4381                : 1.0000 (59/59)
  4413                : 0.8636 (38/44)
  4538                : 1.0000 (69/69)
  4548                : 1.0000 (50/50)
  4554                : 0.8714 (61/70)
  4598                : 0.8857 (62/70)
  4608                : 1.0000 (69/69)
  4627                : 1.0000 (54/54)
  4628                : 0.7692 (10/13)
  4773                : 1.0000 (33/33)
  4781                : 1.0000 (29/29)
  4783                : 0.4536 (44/97)
  4791                : 0.9889 (89/90)
  4815                : 0.9922 (128/129)
  4816                : 1.0000 (81/81)
  4826                : 1.0000 (92/92)
  4838                : 1.0000 (70/70)
  4845                : 0.1803 (11/61)
  4856                : 1.0000 (66/66)
  4872 

📈 Loss: 0.7380 - Val Acc: 0.9855
📊 Per-Class Accuracy:
  1374                : 0.9605 (73/76)
  3875                : 0.9524 (80/84)
  4142                : 1.0000 (46/46)
  4161                : 1.0000 (52/52)
  4307                : 0.6000 (21/35)
  4381                : 0.9831 (58/59)
  4413                : 1.0000 (44/44)
  4538                : 1.0000 (69/69)
  4548                : 1.0000 (50/50)
  4554                : 1.0000 (70/70)
  4598                : 1.0000 (70/70)
  4608                : 0.9855 (68/69)
  4627                : 1.0000 (54/54)
  4628                : 1.0000 (13/13)
  4773                : 1.0000 (33/33)
  4781                : 1.0000 (29/29)
  4783                : 1.0000 (97/97)
  4791                : 0.9778 (88/90)
  4815                : 1.0000 (129/129)
  4816                : 1.0000 (81/81)
  4826                : 1.0000 (92/92)
  4838                : 1.0000 (70/70)
  4845                : 0.9836 (60/61)
  4856                : 0.9848 (65/66)
  4872 

📈 Loss: 0.7147 - Val Acc: 0.9694
📊 Per-Class Accuracy:
  1374                : 0.7237 (55/76)
  3875                : 1.0000 (84/84)
  4142                : 1.0000 (46/46)
  4161                : 0.9615 (50/52)
  4307                : 0.8286 (29/35)
  4381                : 1.0000 (59/59)
  4413                : 1.0000 (44/44)
  4538                : 1.0000 (69/69)
  4548                : 1.0000 (50/50)
  4554                : 1.0000 (70/70)
  4598                : 1.0000 (70/70)
  4608                : 0.9710 (67/69)
  4627                : 1.0000 (54/54)
  4628                : 1.0000 (13/13)
  4773                : 1.0000 (33/33)
  4781                : 1.0000 (29/29)
  4783                : 1.0000 (97/97)
  4791                : 0.9889 (89/90)
  4815                : 1.0000 (129/129)
  4816                : 0.9630 (78/81)
  4826                : 0.9891 (91/92)
  4838                : 1.0000 (70/70)
  4845                : 0.9508 (58/61)
  4856                : 1.0000 (66/66)
  4872 

...

📈 Loss: 0.6709 - Val Acc: 0.9992
📊 Per-Class Accuracy:
  1374                : 1.0000 (76/76)
  3875                : 1.0000 (84/84)
  4142                : 1.0000 (46/46)
  4161                : 1.0000 (52/52)
  4307                : 1.0000 (35/35)
  4381                : 1.0000 (59/59)
  4413                : 1.0000 (44/44)
  4538                : 1.0000 (69/69)
  4548                : 1.0000 (50/50)
  4554                : 1.0000 (70/70)
  4598                : 1.0000 (70/70)
  4608                : 1.0000 (69/69)
  4627                : 1.0000 (54/54)
  4628                : 1.0000 (13/13)
  4773                : 1.0000 (33/33)
  4781                : 1.0000 (29/29)
  4783                : 1.0000 (97/97)
  4791                : 0.9889 (89/90)
  4815                : 1.0000 (129/129)
  4816                : 1.0000 (81/81)
  4826                : 1.0000 (92/92)
  4838                : 1.0000 (70/70)
  4845                : 1.0000 (61/61)
  4856                : 1.0000 (66/66)
  4872 

📈 Loss: 0.6709 - Val Acc: 0.9992
📊 Per-Class Accuracy:
  1374                : 1.0000 (76/76)
  3875                : 1.0000 (84/84)
  4142                : 1.0000 (46/46)
  4161                : 1.0000 (52/52)
  4307                : 1.0000 (35/35)
  4381                : 1.0000 (59/59)
  4413                : 1.0000 (44/44)
  4538                : 1.0000 (69/69)
  4548                : 1.0000 (50/50)
  4554                : 1.0000 (70/70)
  4598                : 1.0000 (70/70)
  4608                : 1.0000 (69/69)
  4627                : 1.0000 (54/54)
  4628                : 1.0000 (13/13)
  4773                : 1.0000 (33/33)
  4781                : 1.0000 (29/29)
  4783                : 1.0000 (97/97)
  4791                : 0.9889 (89/90)
  4815                : 1.0000 (129/129)
  4816                : 1.0000 (81/81)
  4826                : 1.0000 (92/92)
  4838                : 1.0000 (70/70)
  4845                : 1.0000 (61/61)
  4856                : 1.0000 (66/66)
  4872 

✅ Prototypes: (38, 1024) (classes=38)


✅ Tuned cosine threshold τ = 0.950
💾 Saved prototypes + τ to identification_models_KNP\KNP_CONVNEXT_Nighttime_01\prototypes.pt


: 

In [ ]:
# train_and_proto.py
import os
import json
from pathlib import Path
from collections import defaultdict, Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import timm
from helpers.helper import Helper
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
class ProtoInfer:
    def __init__(self, model, device, proto_ckpt_path):
        self.model = model.eval().to(device)
        self.device = device
        ckpt = torch.load(proto_ckpt_path, map_location=device)

        self.prototypes = ckpt["prototypes"].to(device)   # [K,D] L2-normalized
        self.proto_labels = ckpt["proto_labels"]          # list[str]
        self.tau = float(ckpt["tau"])
        emb_dim = ckpt.get("embedding_dim", None)

        # Optional safety check for embedding dimension consistency
        if emb_dim is not None and self.prototypes.shape[1] != emb_dim:
            raise ValueError(f"Embedding dim mismatch: ckpt={emb_dim}, prototypes D={self.prototypes.shape[1]}")

    @torch.no_grad()
    def predict_stack(self, images_tensors, tau=None):
        """
        images_tensors: list[Tensor[C,H,W]] transformed like val_transform
        returns: list[[class_name, count], ...] (includes 'unknown')
        """
        if tau is None:
            tau = self.tau
        #print(tau)
        x = torch.stack(images_tensors).to(self.device)  # [B,C,H,W]
        feats = get_embeddings(self.model, x)            # [B,D]
        sims  = feats @ self.prototypes.T                # [B,K]
        max_sims, idxs = sims.max(dim=1)

        labels = []
        for s, i in zip(max_sims.tolist(), idxs.tolist()):
            print(self.proto_labels[i]," score is ",s)
            if s >= tau:
                labels.append(self.proto_labels[i])
            else:
                labels.append("unknown")

        counts = Counter(labels)
        return [[k, v] for k, v in counts.items()]

# =========================
# EMBEDDINGS / PROTOTYPES
# =========================
@torch.no_grad()
def get_embeddings(model, x):
    """
    Get L2-normalized pre-logits embeddings from ConvNeXtV2.
    timm convnextv2: forward_features -> forward_head(pre_logits=True).
    """
    feats = model.forward_features(x)                  # pre-classifier features
    emb  = model.forward_head(feats, pre_logits=True)  # pooled pre-logits
    return F.normalize(emb, dim=1)  

SAVE_PATH = Path("identification_models_KNP") / "KNP_ConvNeXtV2_fcmae_ft_in22k_in1k_v_RTX8000_Feature_distance_unknown_v_02"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model + weights
NUM_CLASSES = len(json.load(open(SAVE_PATH / "class_mappings.json")))  # dynamic
model = timm.create_model('convnextv2_base.fcmae_ft_in22k_in1k',
                          pretrained=False, num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(SAVE_PATH / "best_model.pth", map_location=device))
model.eval()

infer = ProtoInfer(model, device, proto_ckpt_path=SAVE_PATH / "prototypes.pt")

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])


In [ ]:

image_path = "F:\\Output\\runs\\KUNNEPPU_identification\\KNP_Merge_and_Batch_identification\\1 Sept Report 2025\\reuse tetris_Sumiyoshi-44\\Base_rtx8000_June_15000_v2\\testris_Ch003_Camera 003_E2\\2025-09-02_0514_part 1\\1176\\"
image1 = image_path + "598489_4119_Touching_135_area25717.jpg"
image2 = image_path + "598433_Identifying_Touching_0_area27178.jpg"
# Prepare a stack of images (PIL -> tensor)
pil_images = [Image.open(image1).convert("RGB"),
              Image.open(image2).convert("RGB")]
images_tensors = [val_transform(img) for img in pil_images]

pred_counts = infer.predict_stack(images_tensors,0.9)  # e.g., [['cow_A', 5], ['unknown', 2]]
print(pred_counts)
